In [2]:
from IPython.display import display_html
from sqlalchemy import create_engine
import pandas as pd
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega

# for configuring connection 
from configobj import ConfigObj
import os

from IPython.display import display_html
from sqlalchemy import create_engine

config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5433
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'eicu'

In [3]:
import warnings
warnings.filterwarnings("ignore")


In [5]:
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres
Password: ········


In [6]:
patient = pd.read_sql('SELECT * FROM eicu.patient', con)
#diagnosis = pd.read_sql('SELECT * FROM diagnosis', con)
apache = pd.read_sql('SELECT * FROM eicu.apacheapsvar', con)

result1 = pd.merge(patient,
                 apache,
                 on="patientunitstayid", 
                 how='inner')

In [7]:
hospital = pd.read_sql('Select * from eicu.hospital',con)
print(patient.columns)

Index(['patientunitstayid', 'patienthealthsystemstayid', 'gender', 'age',
       'ethnicity', 'hospitalid', 'wardid', 'apacheadmissiondx',
       'admissionheight', 'hospitaladmittime24', 'hospitaladmitoffset',
       'hospitaladmitsource', 'hospitaldischargeyear',
       'hospitaldischargetime24', 'hospitaldischargeoffset',
       'hospitaldischargelocation', 'hospitaldischargestatus', 'unittype',
       'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype',
       'admissionweight', 'dischargeweight', 'unitdischargetime24',
       'unitdischargeoffset', 'unitdischargelocation', 'unitdischargestatus',
       'uniquepid'],
      dtype='object')


In [9]:
# print(labs.columns)
# print(vital.columns)
# print(vital.patientunitstayid.unique())
#print(patient_count.head(50))

In [10]:
result=result1.drop(['patienthealthsystemstayid', 'ethnicity','wardid','apacheadmissiondx','admissionheight','hospitaladmittime24',
        'hospitaladmitoffset','hospitaladmitsource', 'hospitaldischargeyear',
       'hospitaldischargetime24', 'hospitaldischargeoffset',
       'hospitaldischargelocation','unittype',
       'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype',
       'admissionweight', 'dischargeweight', 'unitdischargetime24','meanbp',
       'unitdischargeoffset', 'unitdischargelocation', 'unitdischargestatus',
       'uniquepid','apacheapsvarid', 'intubated','dialysis','meds', 'hematocrit',
       'creatinine', 'albumin','pco2','glucose'],axis = 1)
print(result.columns)

Index(['patientunitstayid', 'gender', 'age', 'hospitalid',
       'hospitaldischargestatus', 'vent', 'eyes', 'motor', 'verbal', 'urine',
       'wbc', 'temperature', 'respiratoryrate', 'sodium', 'heartrate', 'ph',
       'pao2', 'bun', 'bilirubin', 'fio2'],
      dtype='object')


In [11]:
result2 = pd.merge(result,
                 hospital,
                 on="hospitalid", 
                 how='inner')

print(result2.columns)

Index(['patientunitstayid', 'gender', 'age', 'hospitalid',
       'hospitaldischargestatus', 'vent', 'eyes', 'motor', 'verbal', 'urine',
       'wbc', 'temperature', 'respiratoryrate', 'sodium', 'heartrate', 'ph',
       'pao2', 'bun', 'bilirubin', 'fio2', 'numbedscategory', 'teachingstatus',
       'region'],
      dtype='object')


In [12]:
ids = (result.hospitalid.unique())
print(type(ids[0]))

<class 'numpy.int64'>


In [16]:
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

X_temp = result2
X_temp = X_temp[X_temp.hospitaldischargestatus != '']
# X_temp=X_temp.dropna()

# Mean Imputation for Age
X_temp.replace({'age': {'': '0','> 89': '91'}},inplace=True)
X_temp = X_temp.astype({"age": np.int})
X_temp.replace({'age': {0: int(X_temp.age.mean())}},inplace=True)
# print(X_temp.age.unique())

# Removing age below 18
X_temp = X_temp[X_temp.age >= 18]

# Removing records with no gender given
X_temp = X_temp[X_temp.gender != '']

hds = pd.DataFrame(X_temp['hospitaldischargestatus'])
X_temp.drop(['hospitaldischargestatus'],axis=1,inplace=True)

X_cat = X_temp.select_dtypes(exclude=['int64', 'float64'])                                                                                                         
X_dog = X_temp.select_dtypes(include=['int64', 'float64'])                                                                                                         

X_cat = pd.get_dummies(X_cat)                                                                                                                                 

X_temp=pd.concat([X_cat,X_dog,hds],axis=1)

hds = pd.DataFrame(X_temp.hospitaldischargestatus.map({'Alive': 1, 'Expired': 0}))
X_temp['hospitaldischargestatus'] = hds['hospitaldischargestatus']

X_alive = X_temp.loc[X_temp['hospitaldischargestatus'] == 1]
X_expired = X_temp.loc[X_temp['hospitaldischargestatus'] == 0]

print(X_alive.shape,X_expired.shape)

# X_alive_restricted = X_alive.tail(16000)

# X_temp = X_alive_restricted.append(X_expired)

X = pd.DataFrame(X_temp.drop(['patientunitstayid'],axis=1))
Y = pd.DataFrame(X_temp['hospitaldischargestatus'])                
X = pd.DataFrame(X.drop(['hospitaldischargestatus'],axis=1))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, stratify=Y)

log_regressor   = LogisticRegression(penalty='l1', max_iter = 1e8, class_weight='balanced')
# log_regressor = GridSearchCV(estimator=log_regressor, param_grid={'C':np.logspace(-4,4,10)}, cv= 5)
log_regressor.fit(X_train, y_train)

pred =pd.DataFrame(log_regressor.predict(X_test),columns=["a"])
log_f1_score=(f1_score(y_test,pred))

print(confusion_matrix(y_test,pred))
print(result2.columns)
print(log_regressor.coef_)

print(log_f1_score)
auroc = roc_auc_score(y_test, pred)
print(auroc)

(153037, 32) (16073, 32)
[[ 2355   860]
 [ 7105 23502]]
Index(['patientunitstayid', 'gender', 'age', 'hospitalid',
       'hospitaldischargestatus', 'vent', 'eyes', 'motor', 'verbal', 'urine',
       'wbc', 'temperature', 'respiratoryrate', 'sodium', 'heartrate', 'ph',
       'pao2', 'bun', 'bilirubin', 'fio2', 'numbedscategory', 'teachingstatus',
       'region'],
      dtype='object')
[[ 2.65344872e-02  8.63350598e-01  8.67934635e-01  1.29372595e-01
   0.00000000e+00 -6.14857505e-02 -1.71145598e-01  1.17926485e-01
  -4.38721203e-01  1.42402981e-01 -2.19792860e-01 -9.51413556e-02
  -1.37811304e-01 -3.45028440e-02 -1.14334468e-03 -7.78601938e-01
   4.48665221e-02  2.22828945e-01  9.69131435e-02  1.54078943e-04
  -1.40278467e-02  1.39703814e-02 -1.93596823e-02  7.35603637e-03
  -1.01367462e-02  4.89091441e-02  2.30898919e-03 -1.76570741e-02
  -1.39690487e-01 -1.76851335e-02]]
0.8551001473557823
0.750183724324135


In [17]:
# Leave one hospital out

avg_f1 = 0
avg_auroc = 0
count =0
for i in ids:

    X_train = X_temp[X_temp.hospitalid != i]
    X_test = X_temp[X_temp.hospitalid == i]

    X_train = pd.DataFrame(X_train.drop(['patientunitstayid'],axis=1))
    X_test = pd.DataFrame(X_test.drop(['patientunitstayid'],axis=1))

    y_train = pd.DataFrame(X_train['hospitaldischargestatus'])                
    y_test = pd.DataFrame(X_test['hospitaldischargestatus'])                

    X_train = pd.DataFrame(X_train.drop(['hospitaldischargestatus'],axis=1))
    X_test = pd.DataFrame(X_test.drop(['hospitaldischargestatus'],axis=1))

    
    if not X_test.empty:
        
        count+=1
        log_regressor   = LogisticRegression(penalty='l2', max_iter = 1e8, class_weight='balanced')
        log_regressor.fit(X_train, y_train)

        pred =pd.DataFrame(log_regressor.predict(X_test),columns=["a"])
        log_f1_score=(f1_score(y_test,pred))
        auroc = roc_auc_score(y_test, pred)

        print(confusion_matrix(y_test,pred))
        
        # print(X_train.columns)
        #     print(log_regressor.coef_)

        avg_f1 += log_f1_score
        avg_auroc += auroc
        print(log_f1_score)
        print(auroc)
#         break

print("avg f1 =", avg_f1/count)
print("avg auroc =", avg_auroc/count)

[[ 10   4]
 [ 81 479]]
0.9185043144774688
0.7848214285714286
[[  2   8]
 [ 37 269]]
0.9228130360205832
0.5395424836601307
[[ 354  134]
 [1178 4309]]
0.8678751258811681
0.7553602852644252
[[ 16   8]
 [ 61 253]]
0.88
0.73619957537155
[[ 25  16]
 [131 616]]
0.8934010152284264
0.7171939791687074
[[ 58  37]
 [187 986]]
0.8979963570127505
0.7255530129672005
[[ 28   8]
 [130 652]]
0.9042995839112344
0.8057686842853082
[[  3   7]
 [ 32 193]]
0.908235294117647
0.5788888888888889
[[ 15   2]
 [ 89 396]]
0.8969422423556059
0.8494238932686475
[[ 17   5]
 [ 83 386]]
0.8976744186046511
0.7978774956386897
[[ 246   79]
 [ 998 2206]]
0.8037893969757697
0.7227187169883799
[[ 10   4]
 [ 35 200]]
0.9111617312072893
0.7826747720364742
[[  5   5]
 [ 22 145]]
0.9148264984227129
0.6841317365269461
[[108  45]
 [372 774]]
0.7877862595419847
0.6906375115491222
[[  94   52]
 [ 192 1338]]
0.9164383561643835
0.7591727101799625
[[ 232   91]
 [ 645 1866]]
0.8352730528200538
0.730698240435582
[[ 18  13]
 [ 61 481]]
0.9

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [18]:
from xgboost import XGBClassifier
from keras.utils import to_categorical
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

X_temp1 = result2
X_temp1 = X_temp1[X_temp1.hospitaldischargestatus != '']
# X_temp=X_temp.dropna()

# Mean Imputation for Age
X_temp1.replace({'age': {'': '0','> 89': '91'}},inplace=True)
X_temp1 = X_temp1.astype({"age": np.int})
X_temp1.replace({'age': {0: int(X_temp1.age.mean())}},inplace=True)
# print(X_temp.age.unique())

# Removing age below 18
X_temp1 = X_temp1[X_temp1.age >= 18]

# Removing records with no gender given
X_temp1 = X_temp1[X_temp1.gender != '']

hds = pd.DataFrame(X_temp1.hospitaldischargestatus.map({'Alive': 1, 'Expired': 0}))

X_temp1["hospitaldischargestatus"] = hds["hospitaldischargestatus"]

X_alive = X_temp1.loc[X_temp1['hospitaldischargestatus'] == 1]
X_expired = X_temp1.loc[X_temp1['hospitaldischargestatus'] == 0]

# X_alive_restricted = X_alive.tail(16000)

# X_temp1 = X_alive_restricted.append(X_expired)

X = pd.DataFrame(X_temp1.drop(['patientunitstayid'],axis=1))
Y = pd.DataFrame(X_temp1['hospitaldischargestatus'])                
X = pd.DataFrame(X.drop(['hospitaldischargestatus'],axis=1))

X = pd.DataFrame(X.drop(['numbedscategory'],axis=1))

X_cat = X.select_dtypes(exclude=['int64', 'float64'])                                                                                                         
X_dog = X.select_dtypes(include=['int64', 'float64'])                                                                                                         

X_cat = pd.get_dummies(X_cat)                                                                                                                                 

X=pd.concat([X_cat,X_dog],axis=1)

print(X.head())

# Y = pd.DataFrame(Y.hospitaldischargestatus.map({'Alive': 1, 'Expired': 0}))

# print(X.head())
# print()
negative_class = len(X_alive.index)
positivue_class = len(X_expired.index)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y.iloc[:,0].values)

print(X_train.columns)
s = [100,500,600,900,1250]
loss_dict = {}
weight = positivue_class/negative_class
for i in s:
    xgb_c = XGBClassifier(max_depth=3, n_estimators=i, n_jobs=-1,scale_pos_weight=weight)
    xgb_c.fit(X_train, Y_train,)
    pred = xgb_c.predict(X_test)
    
    print(confusion_matrix(Y_test,pred))
    
    f1 = f1_score(Y_test,pred)
    print(f1)
    auroc = roc_auc_score(y_test, pred)
    print(auroc)
    y = to_categorical(Y_test,2)
    pred = to_categorical(pred,2)
    y=y.astype(np.uint8)
    
    Loss = log_loss(y,pred)
    print(Loss)
    loss_dict[i]= Loss

Using TensorFlow backend.


   teachingstatus  gender_Female  gender_Male  gender_Other  gender_Unknown  \
0           False              1            0             0               0   
1           False              0            1             0               0   
2           False              0            1             0               0   
3           False              0            1             0               0   
4           False              1            0             0               0   

   region_Midwest  region_Northeast  region_South  region_West  age  ...  \
0               1                 0             0            0   70  ...   
1               1                 0             0            0   19  ...   
2               1                 0             0            0   70  ...   
3               1                 0             0            0   75  ...   
4               1                 0             0            0   79  ...   

    wbc  temperature  respiratoryrate  sodium  heartrate   ph  pao2 

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
#  Readmission
plt.figure(figsize = (16,10))
plt.scatter([tree for tree in s], [loss_dict[tree] for tree in s])
plt.plot([tree for tree in s], [loss_dict[tree] for tree in s])
plt.title('Loss for XGBOOST Package')
plt.xlabel('Trees')
plt.ylabel('Loss')
plt.show()


In [ ]:
# retrospective model

diagnosis = pd.read_sql('Select patientunitstayid,icd9code from eicuu.diagnosis limit 100',cnx)
diagnosis_len = len(diagnosis.index)
print(diagnosis_len)

In [ ]:
diagnosis_len = len(diagnosis.index)
dictionary = {}

for i in range(diagnosis_len):
    dictionary[diagnosis.iloc[i][0]]=[]
    
# print(dictionary[141168])

In [ ]:
count=0
for i in range(diagnosis_len):
    count+=1
    print(count)
    codes = diagnosis.iloc[i][1].split(',')
    icd9 = codes[0]
    icd9= icd9.strip()
    if icd9!="" and icd9[0].isdigit():
        dictionary[diagnosis.iloc[i][0]].append(icd9)

In [ ]:
print(dictionary)

In [ ]:
df = pd.DataFrame(list(dictionary.items()), columns=['patientunitstayid', 'icd9'])

df.to_csv("patient_icd.csv")

In [ ]:
print(df.tail(200))

In [ ]:
# df.to_dict("list")
d = {}
for i in df['patientunitstayid'].unique():
    d[i] = [df['icd9'][j] for j in df[df['patientunitstayid']==i].index]

In [ ]:
print(d[141168][0])